In [ ]:
#Connect to AML workspace
from azureml.core import Workspace

ws = Workspace.from_config()

In [ ]:
#Select AML Compute Cluster
cpu_cluster_name = 'amlcluster'

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found an existing cluster, using it instead.')
except ComputeTargetException:
    #When creating a new cluster, use the identity_type argument to attach a system- or user-assigned managed identity
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=0,
                                                           max_nodes=10,
                                                           identity_type='SystemAssigned')
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
#Create Experiment
from azureml.core import Experiment
experiment = Experiment(ws, 'managed-identity-test')

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data import OutputFileDatasetConfig

# create a new runconfig object
test_kv_run_config = RunConfiguration()

# enable Docker 
test_kv_run_config.environment.docker.enabled = True

# set Docker base image to the default CPU-based image
test_kv_run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE

# use conda_dependencies.yml to create a conda environment in the Docker image for execution
test_kv_run_config.environment.python.user_managed_dependencies = False

test_kv_run_config.environment.python.conda_dependencies.add_pip_package('azure-keyvault')

In [ ]:
#Create PythonScriptStep to gather data from remote source and register as AML dataset
test_kv_access_step = PythonScriptStep(
    script_name="test_kv_access.py", 
    compute_target=cpu_cluster, 
    source_directory='.',
    allow_reuse=False,
    runconfig=test_kv_run_config
)


In [ ]:
pipeline = Pipeline(workspace=ws, steps=[test_kv_access_step])
run = experiment.submit(pipeline)
run.wait_for_completion(show_output=True)